In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from PIL import Image
import tensorflow as tf
import cv2 as cv
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import os
import os.path
import shutil
import random
import pickle

Using TensorFlow backend.


# Location of data and the preliminary details of the model (user entered data) 
Enter the file path that contains all the assorted images.  
Also enter the file location of the CSV file that contains the file name, bounding box details, and the class it belongs to  
Enter the size to be resized to 

In [0]:
orig_path = ['location/to/image/folder','location/to/annotated/csv/file']
train_path = 'location/of/train' 
test_path = 'location/of/test'
size = [32,32,3]
split_size = 0.85

# All the necessary functions 

## To get all the subfolders of the given dataset folder

In [0]:
def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in dirs:
            r.append(os.path.join(root, name))
    return r

## To resize the images to the valid format 

In [0]:
def create_data(dataset,size):
    rows,cols,dims = size
    dataset = (dataset.resize([rows,cols],Image.BILINEAR))
    dataset = list(dataset.getdata())
    dataset = np.array(dataset)
    dataset = np.resize(dataset,size)
    dataset = dataset.reshape(size[0],size[1],size[2])
    dataset = dataset.astype('float32')
    dataset = dataset/255
    return dataset

## Read the images and find the corresponding details of them

In [0]:
def get_data(file_path,size):
    target_path = os.path.abspath(file_path)
    
    img_name = []
    data = []
    for img in os.listdir(target_path):
        scriptpath = os.path.dirname(img)
        os.chdir(target_path)
        
        # Read in the image
        image = os.path.join(scriptpath, img)
        image_new = Image.open(image)
        
        #normalise image to our requirements
        image = create_data(image_new,size)
        img_name.append(img)
        data.append(image)
    return data,img_name

## To relocate the images into folders based on their class and splitting them into test and training set
Help in checking for data mismatch

In [0]:
def relocate(orig,new,details,classes,t_split):
    os.chdir(new)
    temp = []
    train_img = []
    train_label = []
    test_img = []
    test_label = []
    create = [new+'train/',new+'test/']
    for i in create:
        if not os.path.exists(i):
            os.mkdir(i)
    for i in range(len(classes)):
        temp.append(new+str(classes[i]))
        ch = new+str(classes[i])
        if not os.path.exists(ch):
            os.mkdir(ch)
    os.chdir(orig)
    for i in details:
        name = os.path.join(orig,i[0])
        for j in temp:
            val = j.split('/')
            val = val[-1]
            if val == i[5]:
#                 shutil.move(name,j)   #to move them permenantly
                shutil.copy(name,j)   #to copy them 
    os.chdir(orig)
    
    
# **************************** FOR MAKING TEST AND TRAIN SPLIT MANUALLY************************
    for i in temp:
        x = [i + '/train',i + '/test']
        if not os.path.exists(x[0]):
            os.mkdir(x[0])
        if not os.path.exists(x[1]):
            os.mkdir(x[1])
        files = [f for f in os.listdir(i) if os.path.isfile(os.path.join(i, f))]
        for j in range(t_split):
            shutil.copy(files[j],x[0])
        del files[0:t_split]
        for j in range(len(files)):
            shutil.copy(files[j],x[1])

# Flattening out in the below steps is not exactly necessary and can be avoided but no harm done. It is a redundant step            

    for i in temp:
        a = list_files(i)
        for j in a:
            b = j.split('/')
            b = b[-1]
            if b == 'train':
                 for img in os.listdir(j):
                        scriptpath = os.path.abspath(img)
                        new_image = (Image.open(scriptpath))
#                         new_image = cv.imread(scriptpath)
                        image = create_data(new_image,size)
                        train_img.append(image)
                        for k in range(len(details)):
                            if details[k][0] == img:
                                train_label.append([details[k][1:6]])
#                                 train_label.append(i.split('_')[-1])
            else:
                for img in os.listdir(j):
                        scriptpath = os.path.abspath(img)
                        new_image = (Image.open(scriptpath))
#                         new_image = cv.imread(scriptpath)
                        image = create_data(new_image,size)
                        test_img.append(image)
                        for k in range(len(details)):
                            if details[k][0] == img:
                                test_label.append([details[k][1:6]])
#                                 test_label.append(i.split('_')[-1])

    for i in temp:
        check = list_files(i)
        for j in check:
            val = j.split('/')
            cval = val[-2]
            val = val[-1]
            if(val == 'train'):
                name = os.listdir(j)
                new_val = os.path.join(create[0],cval)
                if not os.path.exists(new_val):
                    os.mkdir(new_val)
                for k in name:
                    shutil.copy(os.path.join(orig,k),new_val)
            else:
                name = os.listdir(j)
                new_val = os.path.join(create[1],cval)
                if not os.path.exists(new_val):
                    os.mkdir(new_val)
                for k in name:
                    shutil.copy(os.path.join(orig,k),new_val)
    for i in temp:
        shutil.rmtree(i)

## To compile the details of file name, bounding box details, class name of the different Images and then return the training and test set

In [0]:
def compile_data(image_dir,annotated_path,size = [64,64,3]):
    import numpy as np
    import pandas as pd
    import re
    data = pd.read_csv(annotated_path)
    data_list = data['filename'].tolist()
    boundingboxes = []
    t_label = []
    list_data = []
    for i in data['region_shape_attributes']:
        boundingboxes.append(re.findall('\d+', i))
        
    for j in data['region_attributes']:
        temp = j.split(':')
        t_label.append((temp[2].split(',')[0])[1:len(temp[2].split(',')[0])-1])
    
    new = []
    for i in range(len(data_list)):
        new.append([data_list[i]]+boundingboxes[i])
    t_path = image_dir
    
    t_image = []
    t_details = []
    img,details = get_data(t_path,size)
    for i in range(len(img)):
        t_image.append(img[i])
        t_details.append(details[i])
    for i in range(len(t_image)):
        if new[i][0] in t_details:
            list_data.append(new[i]+[(t_label[i])])
#             print(new[i])
    classes = set(t_label)
    classes = list(classes)
    total_no_img = np.size(t_label)
    train_size_per_class = int((total_no_img*split_size)/len(classes))
    
    relocate(orig_path[0],new_path,list_data,classes,train_size_per_class)
    return classes

## Make the data as CNN readable and convert it to matrix of defined size

In [0]:
classes = compile_data(orig_path[0],orig_path[1],size)
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=classes, batch_size=3)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=classes, batch_size=3)
print(train_batches)

Found 12 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


## We now save the data into pickle to avoid calculating them again

In [0]:
os.chdir(new_path)

pickle_out = open("x_train.pickle","wb")
pickle.dump(x_train, pickle_out)
pickle_out.close()

pickle_out = open("y_train.pickle","wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

pickle_out = open("z_train.pickle","wb")
pickle.dump(z_train, pickle_out)
pickle_out.close()